In [ ]:
#Uncomment these if running in colab
!pip install wget
!pip install newspaper3k
!pip install xmltodict
!pip install pandarallel
!pip install datefinder
!pip install pydrive
!pip install selenium
!pip3 install "urllib3 <=1.26.15"
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# <font color='red'>If imports give you any errors, uncomment the install cell above.</font>
### You only need to install these libraries once.
### Future runs or scripts, you can comment them out to save time.

In [1]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from multiprocessing import Pool
import random
from urllib.parse import quote
import pytz




# <font color='red'>The next cell you need to update for each site you scrape </font>
# <font color='red'>I recommend creating a sperate script for each site you scrape. </font>
# <font color='red'>Test scraping of few pages, to check for errors, before scraping all site </font>


### > Name the script above from HPC NEW Scraper to the site name.

### > Please remember to fill/update news outlet and country for site

### > Remember to change "types" depending on page if it's numbered by pages, or click more or scroll down [0,1,2]
### > max_click_SHOW_MORE = the number of clicks or scroll down you like.


### > extract_text_sleep = sleep amount for extracting text
### > extract_urls_sleep = sleep amount for extracting links

### > Remmember to update page identifer on line 39 if website is a pages site






In [25]:
t0 = time.time()

links_file_path = "/xdisk/josorio1/aconverse/scraping/links/"
csv_file_path = "/xdisk/josorio1/aconverse/scraping/csv/"

#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = "Noche y Niebla"
country = "Colombia"
max_click_SHOW_MORE = 500
host = "https://base.nocheyniebla.org"
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
# Allow http and https links
if re.match("^https?.*", host):
  relevant_path = f'^https?:\/\/{host[8:]}.*$'
else:
  relevant_path = f'^https?:\/\/{host}.*$'
urls = ['/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=', \
       '/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2021&filtro%5Bfechaini%5D=01%2FEne%2F2021&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=', \
       '/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2022&filtro%5Bfechaini%5D=01%2FEne%2F2022&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=', \
       '/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=']
urls = [host + url for url in urls]
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more','scroll_down']
type_of_page = types[0]
extract_text_sleep = random.randint(12,24)
extract_urls_sleep = random.randint(12,24)


# If page type is click_more or scroll_down i.e. can get more article by clicking a button with text 'click more' or simply scrolling down
#-----------------------------------------------------------------------------------------------------------------------------------------
xpath_for_link = ""
xpath_for_click_more = ""
#-----------------------------------------------------------------------------------------------------------------------------------------


# If page type is page
#-----------------------------------------------------------------------------------------------------------------------------------------
target_tag = ''
target_tag_class = ""

page_identifier = ''

if type_of_page == 'page':
    urls = [url + page_identifier for url in urls]
pages_each_category = [66, 56, 48, 43] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 500
#-----------------------------------------------------------------------------------------------------------------------------------------

#Get amount of pages per category not having text code
#-----------------------------------------------------------------------------------------------------------------------------------------
retrieve_amount_of_pages_no_text = False
#-----------------------------------------------------------------------------------------------------------------------------------------


#Get amount of pages per category with text code
#-----------------------------------------------------------------------------------------------------------------------------------------
retrieve_amount_of_pages_w_text = False
xpath_for_amount_of_pages = ""
index_for_amount_of_pages_location = 0
#-----------------------------------------------------------------------------------------------------------------------------------------

# in case multi tags
target_tags = []
target_tag_classes = []



In [26]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [27]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)

elif type_of_page == 'click_more' or type_of_page == 'scroll_down':
    for url in urls:
        pages += retrieve_links_from_list(url, True if type_of_page == 'scroll_down' else False)


df = pd.DataFrame(pages, columns = ['page'])
df

,page
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1
1,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=2
2,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=3
3,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=4
4,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=5
...,...
208,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=39
209,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filt

In [28]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        req = request.Request(url, headers={'User-Agent' : "Chrome"})
        con = request.urlopen(req)
        time.sleep(extract_urls_sleep)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None

    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None


    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [29]:
#New Code caused issues

def extract_table_data(url):
    
    order_of_columns = ["fecha","ubiaciones", "personas_responsables", \
              "tipificacion", "victimas", "descripcion", "acciones"]
    
    result = []
    
    soup = getSoup(url)
    try:
        if soup:
            rows = soup.find_all("tr")
            for i, row in enumerate(rows):
                row_data = []
                cells = row.find_all("td")
                for cell in cells:
                    row_data.append(cell.text.strip())
                result.append(row_data)

        else:
            print(f"Invalid URL: {url}")
    except Exception as e:
        print(f"Error with {url}: {e}")
    finally:
        return result

def extract_text_from_df(df):
    content = df['page'].map(lambda x: extract_table_data(x))
    df['content'] = content
    return df

In [30]:
df

,page
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1
1,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=2
2,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=3
3,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=4
4,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=5
...,...
208,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=39
209,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filt

In [31]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [32]:
df

,page
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1
1,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=2
2,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=3
3,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=4
4,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=5
...,...
208,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=39
209,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2023&filtro%5Bfechaini%5D=01%2FEne%2F2023&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filt

In [33]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 10

res = []
while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 10) #NUMBER OF CORES HERE
    res.append(test_df)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))

    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit

df = pd.concat(res)

Batch of data of row range 0-10 complete in 55.99 seconds
4.69% complete
Batch of data of row range 10-20 complete in 55.8 seconds
9.39% complete
Batch of data of row range 20-30 complete in 55.68 seconds
14.08% complete
Batch of data of row range 30-40 complete in 55.99 seconds
18.78% complete
Batch of data of row range 40-50 complete in 55.64 seconds
23.47% complete
Batch of data of row range 50-60 complete in 55.94 seconds
28.17% complete
Batch of data of row range 60-70 complete in 55.77 seconds
32.86% complete
Batch of data of row range 70-80 complete in 56.22 seconds
37.56% complete
Batch of data of row range 80-90 complete in 55.67 seconds
42.25% complete
Batch of data of row range 90-100 complete in 55.68 seconds
46.95% complete
Batch of data of row range 100-110 complete in 55.99 seconds
51.64% complete
Batch of data of row range 110-120 complete in 55.83 seconds
56.34% complete
Batch of data of row range 120-130 complete in 59.36 seconds
61.03% complete
Batch of data of row r

# <font color='red'> If you're having problems extracting links (0 / 40 text retrieved) </font>

### Double check in you browser the articles links you extracted above in 'link' column to see if they're working.
### If they're not, correct your host you filled in above. You might have an extra '/' or repeated text in links such as '/es/es/news'

## <font color='red'> If links are correct </font>
### > Increase the sleep time of extract_urls_sleep in set up cell.
### > So if it was random.randint(5,7), set it to random.randint(10,12)


## <font color='red'> if issue still persist </font>
### > Try to reduce number of cores in above cell from 94 to a lower number
### For example, if you're getting 0 / 40 errors with 94 cores, lower it down to 50. If that doesn't work 25 and so on!

# <font color='red'> If all that doesn't work and site is blocking you, just move on to next site and list reason/issue in excel file :) </font>


In [34]:
df = df.explode("content")

In [35]:
df

,page,content
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,[]
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,"[2020-01-01, Chocó / El Litoral del San Juan, Paramilitares, D:1:706 COLECTIVO AMENAZADO, COLECTIVO COMUNIDADES AFROCOLOMBIANAS BAJO SAN JUAN, COMUNIDADES DEL BAJO CALIMA, En una alerta emitida por la Comisión Intereclesial de Justicia y Paz se denuncia la llegada de cinco hombres armados que se presentaron como paramilitares el pasado 1 de enero, en el muelle de Docordó, cabecera municipal del Litoral San Juan, Chocó. Los armados advirtieron que iniciarían el control territorial, ello pese a que el muelle se encuentra a 150 mts. de la plaza central de Docordó, donde horas antes se había p..."
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,"[2020-01-01, Cauca / Guapi, ELN, D:4:701 HOMICIDIO INTENCIONAL DE PERSONA PROTEGIDA , B:2:41 SECUESTRO, D:4:72 TORTURA, CRISTIAN DAVID CAICEDO, N N, Voceros de la Coordinación de Consejos Comunitarios y Organizaciones de Base del Pueblo Negro de la Costa Pacífica del Cauca-Cococauca denunciaron que: “DOS PERSONAS ASESINADAS, DOS HERIDAS Y UNA RETENIDA EN GUAPI: 2019 TERMINÓ Y 2020 CONTINUÓ VIOLENTO EN LA COSTA PACÍFICA DEL CAUCA. COCOCAUCA: Por mucho que los guapireños lo esperaban en tiempos de fiesta y feliz compartir, la violencia en el pueblo de la costa pacífica del Cauca no cesó en ..."
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,"[2020-01-01, Cauca / Guapi, Sin Información, B:2:40 ASESINATO, BENJAMIN BANGUERA, Voceros de la Coordinación de Consejos Comunitarios y Organizaciones de Base del Pueblo Negro de la Costa Pacífica del Cauca-Cococauca denunciaron que: “Siendo las 8:10 pm fue asesinado a tiros Benjamín Banguera en el barrio El Pueblito, casco urbano de Guapi, también conocido como Mapita. Benjamín era excombatiente del Frente 29 de las Farc que había asumido con total compromiso su apoyo f

In [36]:
df[["fecha","ubiaciones", "personas_responsables", "tipificacion", "victimas", "descripcion", "acciones"]] = pd.DataFrame(df.content.tolist(), index= df.index)
df

,page,content,fecha,ubiaciones,personas_responsables,tipificacion,victimas,descripcion,acciones
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,[],None,None,None,None,None,None,None
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,"[2020-01-01, Chocó / El Litoral del San Juan, Paramilitares, D:1:706 COLECTIVO AMENAZADO, COLECTIVO COMUNIDADES AFROCOLOMBIANAS BAJO SAN JUAN, COMUNIDADES DEL BAJO CALIMA, En una alerta emitida por la Comisión Intereclesial de Justicia y Paz se denuncia la llegada de cinco hombres armados que se presentaron como paramilitares el pasado 1 de enero, en el muelle de Docordó, cabecera municipal del Litoral San Juan, Chocó. Los armados advirtieron que iniciarían el control territorial, ello pese a que el muelle se encuentra a 150 mts. de la plaza central de Docordó, donde horas antes se había p...",2020-01-01,Chocó / El Litoral del San Juan,Paramilitares,D:1:706 COLECTIVO AMENAZADO,"COLECTIVO COMUNIDADES AFROCOLOMBIANAS BAJO SAN JUAN, COMUNIDADES DEL BAJO CALIMA","En una alerta emitida por la Comisión Intereclesial de Justicia y Paz se denuncia la llegada de cinco hombres armados que se presentaron como paramilitares el pasado 1 de enero, en el muelle de Docordó, cabecera municipal del Litoral San Juan, Chocó. Los armados advirtieron que iniciarían el control territorial, ello pese a que el muelle se encuentra a 150 mts. de la plaza central de Docordó, donde horas antes se había posesionado el Alcalde en medio de presencia de la Infantería de Marina y Policía Nacional. Adicionalmente, la Organización afirmó que «los presuntos paramilitares anunciaro...",
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,"[2020-01-01, Cauca / Guapi, ELN, D:4:701 HOMICIDIO INTENCIONAL DE PERSONA PROTEGIDA , B:2:41 SECUESTRO, D:4:72 TORTURA, CRISTIAN DAVID CAICEDO, N N, Voceros de la Coordinación de Consejos Comunitarios y Organizaciones de Base del Pueblo Negro de la Costa Pacífica del Cauca-Cococauca denunciaron que: “DOS PERSONAS ASESINADAS, DOS HERIDAS Y UNA RETENIDA EN GUAPI: 2019 TERMINÓ Y 2020 CONTINUÓ VIOLENTO EN LA COSTA PACÍFICA DEL CAUCA. COCOCAUCA: Por mucho que los guapireños lo esperaban en tiempos de fiesta y feliz compartir, la violencia en el pueblo de la costa pacífica del Cauca no cesó en ...",2020-01-01,Cauca / Guapi,ELN,"D:4:701 HOMICIDIO INTENCIONAL DE PERSONA PROTEGIDA , B:2:41 SECUESTRO, D:4:72 TORTURA","CRISTIAN DAVID CAICEDO, N N","Voceros de la Co

In [37]:
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)
df.dropna(subset=['descripcion'], inplace=True)

In [38]:
df

,page,fecha,ubiaciones,personas_responsables,tipificacion,victimas,descripcion,acciones
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,2020-01-01,Chocó / El Litoral del San Juan,Paramilitares,D:1:706 COLECTIVO AMENAZADO,"COLECTIVO COMUNIDADES AFROCOLOMBIANAS BAJO SAN JUAN, COMUNIDADES DEL BAJO CALIMA","En una alerta emitida por la Comisión Intereclesial de Justicia y Paz se denuncia la llegada de cinco hombres armados que se presentaron como paramilitares el pasado 1 de enero, en el muelle de Docordó, cabecera municipal del Litoral San Juan, Chocó. Los armados advirtieron que iniciarían el control territorial, ello pese a que el muelle se encuentra a 150 mts. de la plaza central de Docordó, donde horas antes se había posesionado el Alcalde en medio de presencia de la Infantería de Marina y Policía Nacional. Adicionalmente, la Organización afirmó que «los presuntos paramilitares anunciaro...",
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,2020-01-01,Cauca / Guapi,ELN,"D:4:701 HOMICIDIO INTENCIONAL DE PERSONA PROTEGIDA , B:2:41 SECUESTRO, D:4:72 TORTURA","CRISTIAN DAVID CAICEDO, N N","Voceros de la Coordinación de Consejos Comunitarios y Organizaciones de Base del Pueblo Negro de la Costa Pacífica del Cauca-Cococauca denunciaron que: “DOS PERSONAS ASESINADAS, DOS HERIDAS Y UNA RETENIDA EN GUAPI: 2019 TERMINÓ Y 2020 CONTINUÓ VIOLENTO EN LA COSTA PACÍFICA DEL CAUCA. COCOCAUCA: Por mucho que los guapireños lo esperaban en tiempos de fiesta y feliz compartir, la violencia en el pueblo de la costa pacífica del Cauca no cesó en diciembre, ni para año nuevo: Entre el 18 de diciembre de 2019 y el 1 enero de 2020 se registraron dos personas asesinadas con arma de fuego, dos heri...",
0,https://base.nocheyniebla.org/casos?commit=Filtrar&filtro%5Bapellidos%5D=&filtro%5Bdepartamento_id%5D=&filtro%5Bdescripcion%5D=&filtro%5Bfechafin%5D=31%2FDic%2F2020&filtro%5Bfechaini%5D=01%2FEne%2F2020&filtro%5Binc_fecha%5D=1&filtro%5Binc_memo%5D=1&filtro%5Binc_presponsables%5D=1&filtro%5Binc_tipificacion%5D=1&filtro%5Binc_ubicaciones%5D=1&filtro%5Binc_victimas%5D=1&filtro%5Bnombres%5D=&filtro%5Borden%5D=fecha&filtro%5Bpresponsable_id%5D%5B%5D=&filtro%5Bq%5D=&filtro%5Brangoedad_id%5D=&filtro%5Bsectorsocial_id%5D=&filtro%5Bsexo%5D=&filtro%5Btviolencia_id%5D=Z&pagina=1,2020-01-01,Cauca / Guapi,Sin Información,B:2:40 ASESINATO,BENJAMIN BANGUERA,"Voceros de la Coordinación de Consejos Comunitarios y Organizaciones de Base del Pueblo Negro de la Costa Pacífica del Cauca-Cococauca denunciaron que: “Siendo las 8:10 pm fue asesinado a tiros Benjamín Banguera en el barrio El Pueblito, casco urbano de Guapi, también conocido como Mapita. Benjamín era excombatiente del Frente 29 de las Farc que había asumido con total compromiso su apoyo frente a los acuerdos de paz firmados. Benjamín vivía en situación de amenaza permanente, ya que los actores armados en la zona intentaban reclutarle, mientras él manifestaba su compromiso pleno

# <font color='red'> IMPORTANT - UPDATE LINE 5 NEXT CELL WITH YOUR "csv" SAVE DIRECTORY </font>

## Update this Path below to reflect your special workspace '/xdisk/josorio1/conflibert_esp/Summer_2023/Sultan/csv/'

## >> For SPANISH <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_esp/Summer_2023
### Find your directory and create folder named 'csv' inside of it.
### Go inside 'csv' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

## >> For Arabic <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_ara/Summer_2023
### Find your directory and create folder named 'csv' inside of it.
### Go inside 'csv' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

In [39]:
#Please update
seconds = time.time() - t0
duration = time.strftime("(%H:%M:%S)",time.gmtime(seconds))

df.to_csv(csv_file_path+duration+'-'+csv_name, encoding = 'utf-8-sig')

print('Time to complete:',duration)

#tz_DFW = pytz.timezone('US/Central')
#current_time = datetime.now(tz_DFW)
#time = current_time.strftime("(%H:%M:%S)")



#df.to_csv(time+csv_name)

Time to complete: (00:28:43)


In [40]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
